In [1]:
! pip install datasets flair spacy


  Using cached numpy-2.0.2-cp312-cp312-win_amd64.whl.metadata (59 kB)
INFO: pip is looking at multiple versions of transformer-smaller-training-vocab to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ----------- ---------------------------- 0.8/2.6 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\hp\AppData\Local\Temp\pip-install-vt8qegng\sentencepiece_a9deac16e8f2436cb5db4aad2ea9a3ff\setup.py", line 126, in <module>
          subprocess.check_call([
        File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 408, in check_call
          retcode = call(*popenargs, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 389, in call
          with Popen(*popenargs, **kwargs) as p:
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
          self._execute_

In [ ]:
from datasets import load_dataset

# Load the dataset
#Train set: 583 documents /Validation set: 146 documents /Test set: 85 documents
#ORG (Organization): 2,246 mentions/PER (Person): 1,856 mentions/LOC (Location): 883 mentions
dataset = load_dataset("taln-ls2n/Adminset-NER")


import pandas as pd
df = pd.DataFrame(dataset['train'])

print(df.head())


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\datasets--taln-ls2n--Adminset-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administ

                                              tokens  \
0  [fin, Procès-Verbal, Conseil, communautaire, d...   
1  [Monsieur, MORLET, excuse, Monsieur, Christoph...   
2  [Monsieur, MORLET, annonce, le, décès, de, Mon...   
3  [Commentaires, ,, débat, Constatant, qu'il, n'...   
4  [Page, 4, sur, 15, <, page, >, 4, <, /, page, ...   

                                            ner_tags  
0  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
1  [B-PER, I-PER, O, B-PER, I-PER, I-PER, O, O, O...  
2  [B-PER, I-PER, O, O, O, O, B-PER, I-PER, I-PER...  
3  [O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER...  
4  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  


In [5]:

# Save as CSV
df.to_csv("adminset_ner.csv", index=False, encoding="utf-8")

# Save as Parquet (for efficiency)
df.to_parquet("adminset_ner.parquet")

In [10]:
import pandas as pd

# Example mapping based on dataset's integer labels
label_map = {
    0: "O",  
    1: "B-PER", 2: "I-PER",
    3: "B-ORG", 4: "I-ORG",
    5: "B-LOC", 6: "I-LOC"
}

# Load dataset (Assuming df is already loaded with "tokens" and "ner_tags" columns)
df = pd.read_parquet("adminset_ner.parquet")


# Save as Flair-format "train.txt"
with open("train.txt", "w", encoding="utf-8") as f:
    for tokens, tags in zip(df["tokens"], df["ner_tags"]):
        for word, tag in zip(tokens, tags):
            f.write(f"{word} {tag}\n")
        f.write("\n")  # separate sentences

print("✅ BIO-tagged dataset successfully converted for Flair training.")


✅ BIO-tagged dataset successfully converted for Flair training.


In [11]:
# Preview first 20 lines of train.txt
with open("train.txt", "r", encoding="utf-8") as f:
    for _ in range(50):
        print(f.readline().strip())


fin O
Procès-Verbal O
Conseil O
communautaire O
du O
lundi O
19 O
juin O
2023 O
L'an O
deux O
mille O
vingt-trois O
, O
le O
lundi O
19 O
juin O
, O
à O
18 O
heures O
30 O
, O
le O
conseil O
communautaire O
s'est O
réuni O
à O
Coucy B-LOC
le I-LOC
Château I-LOC
conformément O
à O
l'article O
2122-17 O
du O
Code O
général O
des O
Collectivités O
Territoriales O
sur O
la O
convocation O
de O
Monsieur B-PER
Vincent I-PER
MORLET I-PER


In [ ]:
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

columns = {0: 'text', 1: 'ner'}

# Load dataset from train.txt
data_folder = "." 
corpus = ColumnCorpus(data_folder, columns, train_file='train.txt')

2025-03-21 15:03:25,294 Reading data from .
2025-03-21 15:03:25,298 Train: train.txt
2025-03-21 15:03:25,302 Dev: None
2025-03-21 15:03:25,306 Test: None
2025-03-21 15:03:27,125 No test split found. Using 10% (i.e. 73 samples) of the train split as test data
2025-03-21 15:03:27,129 No dev split found. Using 10% (i.e. 66 samples) of the train split as dev data


In [7]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus

# Load the pretrained Flair NER model
tagger = SequenceTagger.load("flair/ner-french")


2025-03-21 14:59:17,584 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


In [13]:
trainer = ModelTrainer(tagger, corpus)
trainer.train("flair_ner_output",
              learning_rate=0.05,
              mini_batch_size=16,
              max_epochs=5)

2025-03-21 15:04:37,055 ----------------------------------------------------------------------------------------------------
2025-03-21 15:04:37,061 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      '0-/vol/home-vol2/ml/akbikala/.flair/embeddings/fr-wiki-fasttext-300d-1M'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2348, out_features=2348, bias=True)
  (rnn): LSTM(2348, 256, batch_first=True, bidirectional=True)
  (linea

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\flair\trainers\trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-03-21 15:04:49,645 epoch 1 - iter 3/37 - loss 0.74722724 - time (sec): 12.48 - samples/sec: 250.30 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:05:08,153 epoch 1 - iter 6/37 - loss 0.66627932 - time (sec): 30.99 - samples/sec: 203.88 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:05:19,730 epoch 1 - iter 9/37 - loss 0.57473571 - time (sec): 42.57 - samples/sec: 208.76 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:05:29,523 epoch 1 - iter 12/37 - loss 0.52576980 - time (sec): 52.36 - samples/sec: 216.58 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:05:43,525 epoch 1 - iter 15/37 - loss 0.48540610 - time (sec): 66.36 - samples/sec: 220.19 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:05:55,694 epoch 1 - iter 18/37 - loss 0.44419270 - time (sec): 78.53 - samples/sec: 220.78 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:06:12,028 epoch 1 - iter 21/37 - loss 0.41243059 - time (sec): 94.86 - samples/sec: 215.20 - lr: 0.050000 - momentum: 0.000000
2025-03-21 15:06

100%|██████████| 2/2 [00:20<00:00, 10.05s/it]

2025-03-21 15:07:49,857 DEV : loss 0.16310159862041473 - f1-score (micro avg)  0.3686
2025-03-21 15:07:49,891  - 0 epochs without improvement
2025-03-21 15:07:49,899 saving best model


...

2025-03-21 15:13:10,172 DEV : loss 0.10304801166057587 - f1-score (micro avg)  0.4828
2025-03-21 15:13:10,206  - 0 epochs without improvement


2025-03-21 15:13:10,208 saving best model
2025-03-21 15:13:28,001 ----------------------------------------------------------------------------------------------------
2025-03-21 15:13:28,005 Loading model from best epoch ...
2025-03-21 15:13:35,709 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


100%|██████████| 2/2 [00:17<00:00,  8.98s/it]

2025-03-21 15:13:54,522 
Results:
- F-score (micro) 0.6718
- F-score (macro) 0.6037
- Accuracy 0.5098

By class:
              precision    recall  f1-score   support

         PER     0.9062    0.9255    0.9158        94
         ORG     0.5172    0.3571    0.4225        84
         LOC     0.6500    0.3714    0.4727        35

   micro avg     0.7471    0.6103    0.6718       213
   macro avg     0.6912    0.5514    0.6037       213
weighted avg     0.7107    0.6103    0.6485       213

2025-03-21 15:13:54,524 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6718346253229974}

In [21]:
from flair.models import SequenceTagger
from flair.data import Sentence

tagger = SequenceTagger.load("flair_ner_output/best-model.pt")

print(" Modèle chargé avec succès !")


2025-04-05 03:19:41,960 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>
 Modèle chargé avec succès !


In [22]:

#file_path = r"C:\Users\hp\Downloads\clever contact\cleaned_contract_fixed_final.txt"
#file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract2.txt"
#file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract3.txt"
file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract4.txt"


with open(file_path, "r", encoding="utf-8") as f:
    contract_text = f.read()


sentence = Sentence(contract_text)

#predict 
tagger.predict(sentence)
print(" Résultat de l'anonymisation :")
print(sentence.to_tagged_string())


 Résultat de l'anonymisation :
Sentence[22979]: "SECRETARIAT GENERAL DIRECTION DES FINANCES , DES ACHATS ET DES SERVICES Numéro de consultation : BPM060225 ACCORD-CADRE RELATIF A DES PRESTATIONS DE VALORISATION DES DONNEES CAHIER DES CLAUSES ADMINISTRATIVES PARTICULIÈRES Numéro de consultation : BPM060225 Procédure de passation : Appel doffres ouvert, en application des articles L2124-1, L2124-2, R2124-2 et R2161-2 à R2161-5 du Code de la commande publique ACHETEUR MINISTERES SOCIAUX SECRETARIAT GENERAL DIRECTION DU NUMERIQUE  CCAP BPM060225_DATAVAL BPM060225 2/48 Table des matières 2.6 Forme et étendue de laccord-cadre 10 2.6.1 Forme de chaque lot 10 2.6.2 Etendue de laccord-cadre 11 2.7 Documents contractuels de laccord-cadre 11 2.8 Représentation des parties de laccord-cadre 12 2.8.1 Représentation du pouvoir adjudicateur 12 2.8.2 Représentation du titulaire 12 2.9 Durée de laccord-cadre 12 2.9.1 Cadre général 12 2.9.2 Reconduction de laccord-cadre 12 Chaque lot peut être reconduit 

In [23]:
import json
#  Extract detected entities and store them in a list
extracted_entities = []
for entity in sentence.get_spans("ner"):
    extracted_entities.append({
        "text": entity.text,
        "type": entity.tag,
        "start": entity.start_position,
        "end": entity.end_position
    })

#  Save extracted entities to a JSON file
json_output_file = "flair_finetuned_withnewdataset_entities4.json"
with open(json_output_file, "w", encoding="utf-8") as f:
    json.dump(extracted_entities, f, indent=4, ensure_ascii=False)

In [ ]:
import spacy
from datasets import load_dataset
from spacy.tokens import DocBin

# Load the dataset from Hugging Face
dataset = load_dataset("taln-ls2n/Adminset-NER")

# Load spaCy tokenizer
nlp = spacy.blank("fr")

def create_spacy_doc(tokens, ner_tags):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    current_ent = None

    offset = 0
    for token, tag_label in zip(tokens, ner_tags):
        token_len = len(token)

        if tag_label.startswith("B-"):
            if current_ent:
                ents.append(current_ent)
            current_ent = [offset, offset + token_len, tag_label[2:]]
        elif tag_label.startswith("I-") and current_ent and current_ent[2] == tag_label[2:]:
            current_ent[1] = offset + token_len
        else:
            if current_ent:
                ents.append(current_ent)
                current_ent = None
        offset += token_len + 1  # account for spaces

    if current_ent:
        ents.append(current_ent)

    # Assign entities to the doc
    spacy_ents = []
    for start, end, label in ents:
        span = doc.char_span(start, end, label=label, alignment_mode='expand')
        if span:
            spacy_ents.append(span)

    doc.ents = spacy_ents

    return doc
import os



output_dir = r"C:\Users\hp\Downloads\clever contact"

def convert_to_spacy_format(split, output_dir="."):
    db = DocBin()
    for item in dataset[split]:
        doc = create_spacy_doc(item['tokens'], item['ner_tags'])
        db.add(doc)
    db.to_disk(os.path.join(output_dir, f"{split}.spacy"))
    print(f"{split}.spacy file created in {output_dir}!")

convert_to_spacy_format("train", output_dir=output_dir)
convert_to_spacy_format("validation", output_dir=output_dir)



train.spacy file created in C:\Users\hp\Downloads\clever contact!
validation.spacy file created in C:\Users\hp\Downloads\clever contact!


In [29]:
import spacy
from spacy.cli.train import train

config_str = r"""
[paths]
train = r"C:\Users\hp\Downloads\clever contact\train.spacy"
dev = r"C:\Users\hp\Downloads\clever contact\validation.spacy"

[system]
gpu_allocator = null

[nlp]
lang = "fr"
pipeline = ["tok2vec", "ner"]
batch_size = 64

[components]

[components.tok2vec]
factory = "tok2vec"

[components.ner]
factory = "ner"

[training]
seed = 42
gpu_allocator = null
max_epochs = 10
patience = 1000

[training.optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.001
L2 = 0.01
grad_clip = 1.0

[initialize]
vectors = null
"""

# Write the corrected configuration to file
with open("config.cfg", "w", encoding="utf-8") as f:
    f.write(config_str)

train("config.cfg", output_path="spacy_ner_output",
      overrides={
          "paths.train": r"C:\Users\hp\Downloads\clever contact\train.spacy",
          "paths.dev": r"C:\Users\hp\Downloads\clever contact\validation.spacy"
      })



ℹ Saving to output directory: spacy_ner_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     55.93    0.47    0.79    0.33    0.00
  0     200          0.00   2375.16   14.16   46.30    8.36    0.14
  0     400          0.00   1259.80   20.79   40.00   14.05    0.21
  1     600          0.00   1316.02   28.19   42.57   21.07    0.28
  2     800          0.00   1351.78   43.94   69.57   32.11    0.44
  2    1000          0.00   1342.49   44.24   68.06   32.78    0.44
  3    1200          0.00   1387.31   56.36   67.92   48.16    0.56
  5    1400          0.00   1443.17   54.36   69.07   44.82    0.54
  6    1600          

In [24]:

import spacy
nlp_trained = spacy.load("spacy_ner_output/model-best")




In [25]:
import spacy


#file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract2.txt"
#file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract3.txt"
file_path = r"C:\Users\hp\Downloads\clever contact\cleanedcontract4.txt"
# Load the text file
#file_path = r"C:\Users\hp\Downloads\clever contact\cleaned_contract_fixed_final.txt"

with open(file_path, "r", encoding="utf-8") as f:
    contract_text = f.read()

# Predict entities using spaCy
doc = nlp_trained(contract_text)

# Display results
print("Résultat de l'anonymisation :")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")


Résultat de l'anonymisation :
BPM060225 (LOC)
BPM060225 (LOC)
BPM060225_DATAVAL BPM060225 (PER)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
Monnaie 38 (PER)
BPM060225_DATAVAL BPM060225 (PER)
BPM060225_DATAVAL BPM060225 (PER)
CCAP (ORG)
Département ministériel Partie (ORG)
Direction Générale de la Santé
DGOS (ORG)
CCAP (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
PARIS (LOC)
Business (LOC)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
MOE (ORG)
Direction du Numérique (ORG)
MOE (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
Dataval Charte Accès (PER)
Dataval RGAA (PER)
l'État (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
MCO (LOC)
CCAG-TIC (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
CCAG de références (ORG)
CCAP BPM060225_DATAVAL BPM060225 (ORG)
ICHT (LOC)
l'INSEE (ORG)
l

In [26]:
# Extract entities into a structured list
extracted_entities = []
for ent in doc.ents:
    extracted_entities.append({
        "text": ent.text,
        "type": ent.label_,
        "start": ent.start_char,
        "end": ent.end_char
    })

# Save extracted entities to JSON file
json_output_file = "spacy_finetuned_withnewdataset_entities4.json"
with open(json_output_file, "w", encoding="utf-8") as f:
    json.dump(extracted_entities, f, indent=4, ensure_ascii=False)


In [36]:
import json

def filter_entities_by_suffix(entities, suffix="_2MAINS"):
    filtered_entities = [entity for entity in entities if not entity['text'].endswith(suffix)]
    return filtered_entities


with open("spacy_finetuned_withnewdataset_entities4.json", "r", encoding="utf-8") as f:
    entities = json.load(f)

# Application of the filtre
filtered_entities = filter_entities_by_suffix(entities, "_2MAINS")


with open("filtered_spacy_entities4.json", "w", encoding="utf-8") as f:
    json.dump(filtered_entities, f, indent=4, ensure_ascii=False)

print(" Entités filtrées sauvegardées dans 'filtered_spacy_entities4.json'")


 Entités filtrées sauvegardées dans 'filtered_spacy_entities4.json'


In [32]:
import os
# Change working directory to the correct path
os.chdir(r"C:\Users\hp\Downloads\clever contact")

# Verify the new working directory
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\hp\Downloads\clever contact


In [37]:
import json

def merge_ner_results(spacy_results, flair_results):
    """ Merges SpaCy and Flair results by matching entity text. """
    merged_entities = []
    entity_dict = {}

    # Combine entities from both models
    for entity in spacy_results + flair_results:
        key = entity["text"]  # Match by entity text only

        if key in entity_dict:
            existing = entity_dict[key]
            # If the entity types are the same, keep one
            if entity["type"] == existing["type"]:  
                entity_dict[key] = entity  
        else:
            entity_dict[key] = entity  # Add new entity if it's not present

    return list(entity_dict.values())

# File paths
spacy_json_path = "filtered_spacy_entities4.json"
flair_json_path = "flair_finetuned_withnewdataset_entities4.json"
merged_output_path = "withnewdataset_merged_spacy_flair_entities4.json"

# Load extracted entities from JSON files
with open(spacy_json_path, "r", encoding="utf-8") as f:
    spacy_entities = json.load(f)
print(f" Loaded {len(spacy_entities)} entities from SpaCy output!")

with open(flair_json_path, "r", encoding="utf-8") as f:
    flair_entities = json.load(f)
print(f" Loaded {len(flair_entities)} entities from Flair output!")

# Merge SpaCy & Flair results
final_entities = merge_ner_results(spacy_entities, flair_entities)

# Save merged results to a new JSON file
with open(merged_output_path, "w", encoding="utf-8") as f:
    json.dump(final_entities, f, ensure_ascii=False, indent=4)

print(f"\n Merged NER results saved in {merged_output_path}!")

# Print some merged entities for verification
print("\n Final Merged Entities Sample:")
for entity in final_entities[:10]:  # Print only first 10
    print(f"Entity: {entity['text']} | Type: {entity['type']}")


 Loaded 140 entities from SpaCy output!
 Loaded 42 entities from Flair output!

 Merged NER results saved in withnewdataset_merged_spacy_flair_entities4.json!

 Final Merged Entities Sample:
Entity: BPM060225 | Type: LOC
Entity: BPM060225_DATAVAL BPM060225 | Type: PER
Entity: CCAP BPM060225_DATAVAL BPM060225 | Type: ORG
Entity: Monnaie 38 | Type: PER
Entity: CCAP | Type: ORG
Entity: Département ministériel Partie | Type: ORG
Entity: Direction Générale de la Santé
DGOS | Type: ORG
Entity: PARIS | Type: LOC
Entity: Business | Type: LOC
Entity: MOE | Type: ORG


In [38]:
import json

def get_common_entities(spacy_results, flair_results):
    """Extracts only entities that are common between SpaCy and Flair."""
    common_entities = []
    
    # Convert Flair entities into a dictionary for quick lookup
    flair_dict = { (entity["text"].lower(), entity["type"]) for entity in flair_results }

    # Compare each SpaCy entity to Flair's results
    for entity in spacy_results:
        key = (entity["text"].lower(), entity["type"])  # Normalize case
        if key in flair_dict:  # Entity exists in both SpaCy and Flair
            common_entities.append(entity)

    return common_entities 

# File paths
spacy_json_path = "filtered_spacy_entities4.json"
flair_json_path = "flair_finetuned_withnewdataset_entities4.json"
common_output_path = "withnewdataset_common_spacy_flair_entities4.json"

# Load extracted entities from JSON files
with open(spacy_json_path, "r", encoding="utf-8") as f:
    spacy_entities = json.load(f)
print(f" Loaded {len(spacy_entities)} entities from SpaCy output!")

with open(flair_json_path, "r", encoding="utf-8") as f:
    flair_entities = json.load(f)
print(f" Loaded {len(flair_entities)} entities from Flair output!")

# Find common entities
common_entities = get_common_entities(spacy_entities, flair_entities)

# Save common results to a new JSON file
with open(common_output_path, "w", encoding="utf-8") as f:
    json.dump(common_entities, f, ensure_ascii=False, indent=4)

print(f"\n Common NER results saved in {common_output_path}!")

# Print some common entities for verification
print("\n Common Entities Found in Both SpaCy & Flair:")
for entity in common_entities[:10]:  # Print only first 10
    print(f"Entity: {entity['text']} | Type: {entity['type']}")


 Loaded 140 entities from SpaCy output!
 Loaded 42 entities from Flair output!

 Common NER results saved in withnewdataset_common_spacy_flair_entities4.json!

 Common Entities Found in Both SpaCy & Flair:
Entity: PARIS | Type: LOC
Entity: CCAG-TIC | Type: ORG
Entity: CCAG-TIC | Type: ORG
Entity: CCAG-TIC | Type: ORG
Entity: CCAG-TIC | Type: ORG
Entity: CCAG-TIC | Type: ORG
Entity: Rebeca GOMIS | Type: PER
Entity: ARCHE MC2 | Type: ORG
Entity: Paris | Type: LOC
Entity: PARIS | Type: LOC
